In [21]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import re
import csv
import pandas as pd
import re
import os
from __future__ import division
import numpy as np
import seaborn as sns

%matplotlib ipympl
import matplotlib as mpl
import matplotlib.pyplot as plt

import smtplib
from dateutil import parser
from datetime import date


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import gspread
import google.auth
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError

In [2]:
# contador de execucao
start_time = time.time()


############################
# selenium start
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

############################
# selenium start
table=''

#tenta 5 vezes pegar as informações
for i in range(0,5):
    if i<5 and len(table)!=1:
        try:
            # pegando url
            URL = "https://statusinvest.com.br/acoes/proventos/ibovespa"

            # indices acoes
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
            page = requests.get(URL, headers=headers)
            souptext = BeautifulSoup(page.text)


            # pega todas as tabelas; a que precisa usar é a 3, logo usa-se a 2 abaixo

            driver.get(URL)
            driver.maximize_window()

            ############################
            # clicks para setar data

            #calendario

            driver.implicitly_wait(1)
            #envia data inicial, 2014
            driver.find_element(By.XPATH, "/html/body/main/section/div[2]/form/div/div[3]/div[1]/input").clear()
            driver.implicitly_wait(1)
            driver.find_element(By.XPATH, "/html/body/main/section/div[2]/form/div/div[3]/div[1]/input").send_keys('01/01/2014')


            #todos valores, não apenas 100 ou 200 por pagina
            driver.implicitly_wait(15)
            driver.find_element(By.XPATH, '/html/body/main/section/div[2]/div[1]/div/div/div[2]/div').click()
            driver.implicitly_wait(1)
            driver.find_element(By.XPATH, '/html/body/main/section/div[2]/div[1]/div/div/div[2]/div/ul/li[3]').click()


            time.sleep(15)
            table_inicial = driver.find_element(By.XPATH,'/html/body/main/section/div[2]/div[1]/div/div/div[1]/div[2]/div/table').text

            driver.close()
            
            
            pass


        except:
            print(f'\n\n----------- Tentativas, {i} -----------\n', i)
            break
        
    else:
        pass



----------- Tentativas, 1 -----------
 1


In [58]:
#print(table_inicial)
table=table_inicial
table=table.split()  
table=table[8:]


# removendo strings problematicas
for i in range(0, len(table)-74):
    if 'Ren' in table[i]:
        del table[i]

# listas de manipulacao
acoes=[]
mes_pagamento=[]

# separando dados da tabela a cada X itens
acoes=table[::7]
mes_pagamento=table[4::7]


# deixando apenas os meses
for i in range(0, len(mes_pagamento)):
    mes_pagamento[i]=mes_pagamento[i][3:5]
    

# prints verificacao

#print('tamanho lista acoes: ', len(acoes))
#print(acoes)
#print('tamanho lista pagamentos: ', len(mes_pagamento))
#print(mes_pagamento)


# inserindo dados em dataframe para manipulação em massa
df = pd.DataFrame({'acoes_tickers': acoes,
     'mes_pagamento_div': mes_pagamento
    })

#transformando coluna para numerico
df["mes_pagamento_div"] = pd.to_numeric(df["mes_pagamento_div"])

#today = datetime.date.today()
#df.loc[-1] =  today # adding a row
#df.index = df.index + 1  # shifting index
#df.sort_index(inplace=True) 
#df

#gerando dummies com valores dos meses contados pagados
df_heatmap = df.pivot_table(index='acoes_tickers', columns='mes_pagamento_div', aggfunc='size', fill_value=0)
#df_heatmap



try:
    df_heatmap.to_csv('mapa_dividendos.csv', mode='w', encoding='utf-8')
    print(f'@@@@@@@@@@ Arquivo criado @@@@@@@@@@')
except:
    print("\nEscrita falhou -------------")

df_heatmap


Escrita falhou -------------


mes_pagamento_div,1,2,3,4,5,6,7,8,9,10,11,12
acoes_tickers,,,,,,,,,,,,
ABEV3,5,3,1,1,0,1,3,2,1,0,2,10
ALOS3,0,0,0,0,9,1,2,0,0,0,0,0
ALPA4,0,0,2,7,0,1,4,0,3,3,2,6
ARZZ3,8,0,0,1,4,5,8,2,2,4,3,7
ASAI3,0,1,0,0,0,3,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
VBBR3,2,2,0,3,3,0,1,2,7,1,0,3
VIVA3,0,0,0,0,6,0,0,0,0,0,0,1
VIVT3,0,1,1,12,1,4,15,21,0,6,0,16
